In [1]:
import sympy as sp
import numpy as np

In [2]:
x, y = [sp.IndexedBase(e) for e in ['x', 'y']]
m = sp.symbols('m', integer=True)
a, b = sp.symbols('a b', real=True)
i = sp.Idx('i', m)

In [14]:
loss = (y[i] - (a*x[i] + b))**2

In [15]:
loss

(-a*x[i] - b + y[i])**2

Having defined the loss function using indexed variables we might hope that the
implicit summation of repeated indexes might fall through to derivative
but it looks like this isn't the case.

Below we see taking derivative wrt to fit parameters is only applied
to each point rather than the whole sum, which is incorrect.

In [11]:
sp.solve(loss.diff(a), a)

[(-b + y[i])/x[i]]

In [12]:
sp.solve(loss.diff(b), b)

[-a*x[i] + y[i]]

Try adding explicit summation around the loss expression.  This gives the
correct set of equations for derivatives but a solution can't be found.

In [17]:
sp.diff(sp.Sum(loss, i),a)

Sum(-2*(-a*x[i] - b + y[i])*x[i], (i, 0, m - 1))

In [18]:
sp.diff(sp.Sum(loss, i), b)

Sum(2*a*x[i] + 2*b - 2*y[i], (i, 0, m - 1))

In [19]:
sp.solve([sp.diff(sp.Sum(loss, i),a), sp.diff(sp.Sum(loss, i),b)], [a, b])

[]

In [22]:
sp.solve([loss.expand().diff(a), loss.expand().diff(b)], [a,b])

{a: -b/x[i] + y[i]/x[i]}

MatrixSymbol seems to be the trick

In [84]:
x_2 = sp.MatrixSymbol('x', m, 1)
y_2 = sp.MatrixSymbol('y', m, 1)
a_2 = sp.MatrixSymbol('a', 1, 1)
b_2 = b*sp.OneMatrix(m, 1)

In [87]:
err = y_2 - (x_2*a_2 + b_2)
err

(-b)*1 - x*a + y

In [91]:
objective = (err.T * err)
objective

((-b)*1 - a.T*x.T + y.T)*((-b)*1 - x*a + y)

In [93]:
objective.diff(a_2)

-2*x.T*((-b)*1 - x*a + y)

In [94]:
objective.diff(b)

-((-b)*1 - a.T*x.T + y.T)*1 - 1*((-b)*1 - x*a + y)

Functions of Matrices e.g. generator of rotations

In [ ]:
t = sp.symbols('t', real=True)
g = sp.Matrix([[0, -t], [t, 0]])

In [40]:
g

TypeError: cannot determine truth value of Relational

In [37]:
sp.exp(g)

Matrix([
[cos(t), -sin(t)],
[sin(t),  cos(t)]])

ValueError: m is not an integer